In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json

def ImShow(im, size=(14,17)):
    plt.figure(figsize = size)
    plt.imshow(im, 'gray')
    plt.show()

In [2]:
def specifyRect(roi, x0, w0, y0, h0):
    ImShow(roi)
    cv2.imwrite(str(x0) + '.jpeg', roi)
    thresh = cv2.threshold(roi,127,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
    ImShow(thresh)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(roi, (x, y), (x+w, y+h), (0,255,0), 2)
    ImShow(roi)

In [3]:
def saveRect(roi, x, w, y, h, current_frame):
#    ImShow(roi)
 #   specifyRect(roi, x, w, y, h)
    if current_frame in data:
        data[current_frame].append([x, y, w, h])
    else:
        data[current_frame] = []
        data[current_frame].append([x, y, w, h])
    #with open("ground_truth.txt", "a") as myfile:
        #myfile.write(''.join([str(current_frame), ' : ', str(x), ' ', str(y), ' ', str(w), ' ', str(h), '\n']))

In [4]:
def printRects(frame):
    global rects
    global current_frame
    
    for Pt in rects:
        if len(Pt) == 2:
            y1 = min(Pt[0][1], Pt[1][1])
            y2 = max(Pt[0][1], Pt[1][1])
            x1 = min(Pt[0][0], Pt[1][0])
            x2 = max(Pt[0][0], Pt[1][0])
            roi = frame[y1:y2, x1:x2]
            saveRect(roi, x1, x2-x1, y1, y2-y1, current_frame)

In [5]:
frame = ""
rects = []
cropping = False
data = {}

# draw a rectangle around the region of interest
def add_rect():
    global cloneframe
    global rects
    try:
        for Pt in rects:
            cv2.rectangle(cloneframe, Pt[0], Pt[1], (0, 255, 0), 1)

        if cropping:
            cv2.rectangle(cloneframe, refPt[0], movePoint, (0, 0, 255), 1)
            
    except Exception as e:
        _a = 1
     #   print("Frame error! " + str(e.__class__))


def click_and_crop(event, x, y, flags, param):
    # grab references to the global variables
    global refPt, cropping, movePoint

    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being
    # performed
    if event == cv2.EVENT_MOUSEMOVE:
        if cropping:
            movePoint = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        refPt.append((x, y))
        rects.append(refPt)
        cropping = False

cap = cv2.VideoCapture(r'C:\Users\Yuriy\Dropbox\Project&Practice\HumanDetection\VIDEO_Dataset\video.mp4')

if (cap.isOpened() == False):
    print("Error opening video stream or file")

cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', click_and_crop)

current_frame = -1

while (cap.isOpened()):
    ret, frame = cap.read()
    current_frame += 1
        
#     if (current_frame < 8000):
#         continue
    
#     if (current_frame == 8250):
#         break
    
    stop = False
    
    while(1):
        cloneframe = frame.copy()
        add_rect()
        cv2.imshow('Frame', cloneframe)
        last_key = cv2.waitKey(25) & 0xFF
        if last_key == ord('x'):
            rects = rects[:-1]
        if last_key == ord('n'):
            printRects(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
            with open('data.json', 'w') as outfile:
                json.dump(data, outfile)
            break
        if last_key == ord('q'):
            stop = True
            break
    
    if stop:
        break
        
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)
        
cap.release()
cv2.destroyAllWindows()

In [6]:
with open("data.json", "r") as read_file:
    data = json.load(read_file)
data

{'8000': [[792, 106, 40, 72], [852, 106, 28, 54]],
 '8001': [[792, 106, 40, 72], [852, 106, 28, 54]],
 '8002': [[792, 106, 40, 72], [852, 106, 28, 54]],
 '8003': [[790, 107, 40, 74], [847, 104, 31, 55]],
 '8004': [[790, 107, 40, 74], [847, 104, 31, 55]],
 '8005': [[790, 107, 40, 74], [843, 106, 31, 54]],
 '8006': [[790, 107, 40, 74], [843, 106, 31, 54]],
 '8007': [[787, 107, 43, 71], [839, 104, 32, 54]],
 '8008': [[787, 107, 43, 71], [839, 104, 32, 54]],
 '8009': [[787, 107, 43, 71], [839, 104, 32, 54]],
 '8010': [[781, 105, 45, 71], [834, 101, 35, 57]],
 '8011': [[781, 105, 45, 71], [834, 101, 35, 57]],
 '8012': [[781, 105, 45, 71], [834, 101, 35, 57]],
 '8013': [[781, 105, 45, 71], [834, 101, 35, 57]],
 '8014': [[784, 101, 41, 73], [831, 101, 33, 52]],
 '8015': [[784, 101, 41, 73], [831, 101, 33, 52]],
 '8016': [[784, 101, 41, 73], [831, 101, 33, 52]],
 '8017': [[780, 100, 40, 73], [828, 101, 33, 54]],
 '8018': [[780, 100, 40, 73], [828, 101, 33, 54]],
 '8019': [[780, 100, 40, 73], [

In [11]:
with open("my.json", "r") as read_file:
    data = json.load(read_file)
data

{'7500': [[182, 126, 56, 85]],
 '7501': [[182, 126, 56, 85]],
 '7502': [[182, 126, 56, 85]],
 '7503': [[182, 126, 56, 85]],
 '7504': [[182, 126, 56, 85]],
 '7505': [[182, 126, 56, 85]],
 '7506': [[182, 126, 56, 85]],
 '7507': [[182, 126, 56, 85]],
 '7508': [[182, 126, 56, 85]],
 '7509': [[186, 127, 61, 90]],
 '7510': [[186, 127, 61, 90]],
 '7511': [[186, 127, 61, 90]],
 '7512': [[186, 127, 61, 90]],
 '7513': [[186, 127, 61, 90]],
 '7514': [[186, 127, 61, 90]],
 '7515': [[186, 127, 61, 90]],
 '7516': [[195, 129, 60, 92]],
 '7517': [[195, 129, 60, 92]],
 '7518': [[195, 129, 60, 92]],
 '7519': [[195, 129, 60, 92]],
 '7520': [[195, 129, 60, 92]],
 '7521': [[195, 129, 60, 92]],
 '7522': [[195, 129, 60, 92]],
 '7523': [[195, 129, 60, 92]],
 '7524': [[195, 129, 60, 92]],
 '7525': [[195, 131, 67, 95]],
 '7526': [[195, 131, 67, 95]],
 '7527': [[195, 131, 67, 95]],
 '7528': [[195, 131, 67, 95]],
 '7529': [[195, 131, 67, 95]],
 '7530': [[195, 131, 67, 95]],
 '7531': [[195, 131, 67, 95]],
 '7532':

In [ ]:
C:/Users/Yuriy/Google/VideoData/Frames/video_frame_1000.jpeg

In [26]:
# for i in range(1000, 1250):
#     d = data[str(i)]
#     new_d = []
#     for rect in d:
#         x,y,w,h = rect
#         if (w > 5) and (h > 5):
#             new_d.append(rect)
#     data[str(i)] = new_d

In [14]:
new_data = {}

for im_num, cnts in data.items():
    new_data['C:/Users/Yuriy/Google/VideoData/Frames/video_frame_' + im_num + '.jpeg'] = cnts
    
# for i in range(8233,8250):
#     new_data['C:/Users/Yuriy/Google/VideoData/Frames/video_frame_' + str(i) + '.jpeg'] = []

with open('my.json', 'w') as outfile:
    json.dump(new_data, outfile)

In [35]:
video_path = r'C:\Users\Yuriy\Google\VideoData\video.mp4'
cap = cv2.VideoCapture(video_path)

current_frame = -1

while(cap.isOpened()):
    current_frame += 1
    
    ret, frame = cap.read()
    
    if (current_frame < 1500):
        continue
    
    if (current_frame == 1750):
        break
        
    d = data[str(current_frame)]
    for rect in d:
        x,y,w,h = rect
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
video_path = r'C:\Users\Yuriy\Google\VideoData\video.mp4'
cap = cv2.VideoCapture(video_path)

current_frame = -1

while(cap.isOpened()):
    current_frame += 1
    
    ret, frame = cap.read()
    
    if (current_frame < 7500):
        continue
    
    if (current_frame == 8250):
        break
        
    cv2.imwrite('C:/Users/Yuriy/Google/VideoData/Frames/video_frame_' + str(current_frame) + '.jpeg', frame)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [13]:
video_path = r'C:\Users\Yuriy\Google\VideoData\video.mp4'
cap = cv2.VideoCapture(video_path)

current_frame = -1

while(cap.isOpened()):
    current_frame += 1
    
    ret, frame = cap.read()
    
    if (current_frame < 7500):
        continue
    
    if (current_frame == 8250):
        break
        
    
  #  cv2.putText(frame,str(current_frame), (100,100), cv2.FONT_HERSHEY_PLAIN, 5, 255)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
video_path = r'C:\Users\Yuriy\Google\VideoData\video.mp4'
cap = cv2.VideoCapture(video_path)

current_frame = -1

while(cap.isOpened()):
    current_frame += 1
    
    ret, frame = cap.read()
    
    if (current_frame < 8000):
        continue
    
    if (current_frame == 8250):
        break
        
    
  #  cv2.putText(frame,str(current_frame), (100,100), cv2.FONT_HERSHEY_PLAIN, 5, 255)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()